# I started with this notebook and adapated a bunch of things 

Source: 
https://www.kaggle.com/code1110/numerai-tournament

## This Notebook does the following things:
1. Get  the Training and Tournment data from the Numerai static links. 
2. Cast the data into Panadas DF and limit the memory use. 
3. Train two models and submit the result on numerai.
4. I remove all the internal scoring for the two models. You just pick two Hyperparam sets for Light GBM and then submit them. 



## There are some problems in this. Spend a few hours to refactor the sumbit methods


In [5]:
!pip install numerapi
import numerapi

In [6]:
import numpy as np
import pandas as pd
import os, sys
import gc
import pathlib
import json
from typing import List, NoReturn, Union, Tuple, Optional, Text, Generic, Callable, Dict
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, QuantileTransformer
from sklearn.model_selection import KFold, StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss, mean_squared_error, mean_absolute_error, f1_score
from scipy.stats import spearmanr # -P I think this is corr. 
import joblib

# model
import lightgbm as lgb
import xgboost as xgb
import operator

# visualize
import matplotlib.pyplot as plt
import matplotlib.style as style
import seaborn as sns
from matplotlib import pyplot
from matplotlib.ticker import ScalarFormatter
sns.set_context("talk")
style.use('seaborn-colorblind')



Methods

In [7]:
def cast_eras_as_int(x): # this is used to cast the eras from strings to ints
    try:
        return int(x[3:]) # the eras look like era03
    except:
        return 1000

# unclear if numerapi.download_latest_data() would be faster
def read_data(data='train'):
    # get data 
    if data == 'train':
        df = pd.read_csv('https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_training_data.csv.xz')

        # The test data is significantly larger.
        # test data is the live tournment data
    elif data == 'test':
        df = pd.read_csv('https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_tournament_data.csv.xz')
        
    
    # features
    feature_cols = df.columns[df.columns.str.startswith('feature')]
    
    # map to int, to reduce the memory demand
    mapping = {0.0 : 0, 0.25 : 1, 0.5 : 2, 0.75 : 3, 1.0 : 4} # this is very clever -P
    for c in feature_cols:
        df[c] = df[c].map(mapping).astype(np.uint8)
        
    df["era"] = df["era"].apply(cast_eras_as_int)# also cast era to int
    return df

# Load the Data from Numerai

In [8]:
%%time
# load in the Training data
train_data = read_data('train')
train_data.shape


(501808, 314)
CPU times: user 46.8 s, sys: 3.52 s, total: 50.3 s
Wall time: 54.7 s


In [18]:
train_data.columns

Index(['id', 'era', 'feature_intelligence1', 'feature_intelligence2',
       'feature_intelligence3', 'feature_intelligence4',
       'feature_intelligence5', 'feature_intelligence6',
       'feature_intelligence7', 'feature_intelligence8',
       ...
       'feature_wisdom38', 'feature_wisdom39', 'feature_wisdom40',
       'feature_wisdom41', 'feature_wisdom42', 'feature_wisdom43',
       'feature_wisdom44', 'feature_wisdom45', 'feature_wisdom46', 'target'],
      dtype='object', length=313)

In [13]:
%%time
# the testing data is the tournament data
tournament_data = read_data('test')
tournament_data.shape


CPU times: user 2min 48s, sys: 10.4 s, total: 2min 58s
Wall time: 3min 8s


In [23]:
print(tournament_data['id'].count())
tournament_data.columns
tournament_data['target'].count() # count non nan targets

1676742


137779

Create a seperate valid split. 
Read more https://towardsdatascience.com/train-validation-and-test-sets-72cb40cba9e7
Train: Model sees and learns this data

Validation: Use this to see your score. This is what you use to tune hyperparameters.

Valid is a subset of the test data. it is where the data_type is 'Validation'

### I don't have good evidence for splits in this size. It might be better for some other splits

In [10]:
# %%time
# # validation is derived from the live tournament data
# valid = tournament_data[tournment_data["data_type"] == "validation"].reset_index(drop = True) # when the data_type == Validation that means there is already a target for those vectors.
# # those are the last spot check on the model's vectors 

# print(valid.columns) # valid is a df where
# # validation split
# valid.loc[valid["era"] > 180, "valid2"] = True # Every era after 180 is in validation

# # you will want ot lookat the places where
# valid.loc[valid["era"] <= 180, "valid2"] = False # Every era before is not in the validation set. 


### There is no 'data_type' column when you are submittign out of the window. Today is Tuesday and there is no 'data_type' column. 

come back and finish refactoring this on saturday / Sunday.

In [20]:
# remove data_type to save memory
train_data.drop(columns=["data_type"], inplace=True)
# valid.drop(columns=["data_type"], inplace=True) You don't have validation anymore
tournament_data.drop(columns=["data_type"], inplace=True)

print('The number of records: train {:,}, test {:,}'.format(train_data.shape[0], tournament_data.shape[0])) # df.shape[0] is number of rows.

KeyError: ignored

## Determine features

In [24]:
# features
features = [f for f in tournament_data.columns.values.tolist() if 'feature' in f] # fancy for loop to get all the feature names. -p
print('There are {} features.'.format(len(features)))
print(type(features))

There are 310 features.
<class 'list'>


## Target

In [26]:
target = tournament_data.columns[tournament_data.columns.str.startswith('target')].values.tolist()[0] # 'target'

# Modeling
The example script of xgboost made by numerai [Example](https://github.com/numerai/example-scripts/blob/master/example_model.py)



Link to hyperparameters of xgboost 
https://xgboost.readthedocs.io/en/latest/parameter.html


In [ ]:
# # create a model and fit (公式example)

# untested. -p
# model = xgb.XGBRegressor(max_depth=5, learning_rate=0.01, n_estimators=2000, n_jobs=-1, colsample_bytree=0.1)
# model.fit(train[features], train[target])

# Train mrquants model and tut_model

In [27]:
%%time

mr_quants_parmas = {
            'n_estimators': 5000,
            'objective': 'regression',
            'boosting_type': 'gbdt',
            'max_depth': 55,
            'learning_rate': 0.013, 
            'feature_fraction': 0.095,
            'seed': 42
            }
tut_parmas = {
            'n_estimators': 3000,
            'objective': 'regression',
            'boosting_type': 'gbdt',
            'max_depth': 20,
            'learning_rate': 0.013, 
            'feature_fraction': 0.095,
            'seed': 42
            }

mr_quants_model = lgb.LGBMRegressor(**mr_quants_parmas) 
mr_quants_model.fit(train_data[features], train_data[target])

tut_model = lgb.LGBMRegressor(**tut_parmas) # I think ** 
tut_model.fit(train_data[features], train_data[target])

CPU times: user 17min 13s, sys: 4.83 s, total: 17min 18s
Wall time: 4min 25s


# Feature importance

You can see what features are important and the distribution of weights assigned to each of the features.
You might use this as part of the meta model of light xbgoost 



In [28]:
# %%time
# feature_importance =pd.DataFrame(model.feature_importances_, index=features, columns=['importance'])
# feature_importance.describe()

It might make sense to group your models into clusters based on this stat. 
You would want to scale them down to 1 when you group them. 

# Validation Score
These are the methods to evaluate your model.
I did not write these models, many of the comments written I wrote though.


In [29]:
# naming conventions
PREDICTION_NAME = 'prediction'
TARGET_NAME = target # 'target is the string named 'target'
# EXAMPLE_PRED = 'example_prediction'

# ---------------------------
# Functions
# ---------------------------
def valid4score(valid : pd.DataFrame, pred : np.ndarray, load_example: bool=True, save : bool=False) -> pd.DataFrame:
    """
    Generate new valid pandas dataframe for computing scores
    
    :INPUT:
    - valid : pd.DataFrame extracted from tournament data (data_type='validation')
    
    """
    valid_df = valid.copy() # the validation dataframe you use this to test the CORR and other values

    # Your model creates an array of floats [0,1] rank method converst them in a list of ints. 

    # your lis tof ints is then compared to their list of ints. 
    valid_df['prediction'] = pd.Series(pred).rank(pct=True, method="first") # pred is the array of predictions your model creates for the set of validation vectors.  
    # I am unsure if this preds is a float only only between 0,1,2,3,4. 
    valid_df.rename(columns={target: 'target'}, inplace=True)
    
    # I don't know what the load example boolean is. I think you can use this to save predictions.
    if load_example:
        valid_df[EXAMPLE_PRED] = pd.read_csv(EXP_DIR + 'valid_df.csv')['prediction'].values
    
    if save==True:
        valid_df.to_csv(OUTPUT_DIR + 'valid_df.csv', index=False)
        print('Validation dataframe saved!')
    
    return valid_df

def compute_corr(valid_df : pd.DataFrame):
    """
    Compute rank correlation

    THIS IS WHAT YOU ARE PRIMARILY PAID ON 
    
    :INPUT:
    - valid_df : pd.DataFrame where at least 2 columns ('prediction' & 'target') exist
    
    """
    # this uses Person Correilation. 
    # I You are paid on spearman corrilation. That is where the ratio of change is important not the raw amount of change
    # see: https://support.minitab.com/en-us/minitab-express/1/help-and-how-to/modeling-statistics/regression/supporting-topics/basics/a-comparison-of-the-pearson-and-spearman-correlation-methods/
    return np.corrcoef(valid_df["target"], valid_df['prediction'])[0, 1]

def compute_max_drawdown(validation_correlations : pd.Series):
    """
    Compute max drawdown
    
    :INPUT:
    - validation_correaltions : pd.Series
    """
    
    rolling_max = (validation_correlations + 1).cumprod().rolling(window=100, min_periods=1).max()
    daily_value = (validation_correlations + 1).cumprod()
    max_drawdown = -(rolling_max - daily_value).max()
    
    return max_drawdown

def compute_val_corr(valid_df : pd.DataFrame):
    """
    Compute rank correlation for valid periods
    
    :INPUT:
    - valid_df : pd.DataFrame where at least 2 columns ('prediction' & 'target') exist
    """
    
    # all validation
    correlation = compute_corr(valid_df)
    print("rank corr = {:.4f}".format(correlation))
    return correlation
    
def compute_val_sharpe(valid_df : pd.DataFrame):
    """
    Compute sharpe ratio for valid periods
    
    :INPUT:
    - valid_df : pd.DataFrame where at least 2 columns ('prediction' & 'target') exist
    """
    # all validation
    d = valid_df.groupby('era')[['target', 'prediction']].corr().iloc[0::2,-1].reset_index()
    me = d['prediction'].mean()
    sd = d['prediction'].std()
    max_drawdown = compute_max_drawdown(d['prediction'])
    print('sharpe ratio = {:.4f}, corr mean = {:.4f}, corr std = {:.4f}, max drawdown = {:.4f}'.format(me / sd, me, sd, max_drawdown))
    
    return me / sd, me, sd, max_drawdown
    
def feature_exposures(valid_df : pd.DataFrame):
    """
    Compute feature exposure
    
    :INPUT:
    - valid_df : pd.DataFrame where at least 2 columns ('prediction' & 'target') exist
    """
    feature_names = [f for f in valid_df.columns
                     if f.startswith("feature")]
    exposures = []
    for f in feature_names:
        fe = spearmanr(valid_df['prediction'], valid_df[f])[0]
        exposures.append(fe)
    return np.array(exposures)

def max_feature_exposure(fe : np.ndarray):
    return np.max(np.abs(fe))

def feature_exposure(fe : np.ndarray):
    return np.sqrt(np.mean(np.square(fe)))

def compute_val_feature_exposure(valid_df : pd.DataFrame):
    """
    Compute feature exposure for valid periods
    
    :INPUT:
    - valid_df : pd.DataFrame where at least 2 columns ('prediction' & 'target') exist
    """
    # all validation
    fe = feature_exposures(valid_df)
    fe1, fe2 = feature_exposure(fe), max_feature_exposure(fe)
    print('feature exposure = {:.4f}, max feature exposure = {:.4f}'.format(fe1, fe2))
     
    return fe1, fe2

# to neutralize a column in a df by many other columns
#         I have no idea what this method does. -P. need to read about it and write up a link to it. 
def neutralize(df, columns, by, proportion=1.0):
    scores = df.loc[:, columns]
    exposures = df[by].values

    # constant column to make sure the series is completely neutral to exposures
    exposures = np.hstack(
        (exposures,
         np.asarray(np.mean(scores)) * np.ones(len(exposures)).reshape(-1, 1)))

    scores = scores - proportion * exposures.dot(
        np.linalg.pinv(exposures).dot(scores))
    return scores / scores.std()


# to neutralize any series by any other series
def neutralize_series(series, by, proportion=1.0):
    scores = series.values.reshape(-1, 1)
    exposures = by.values.reshape(-1, 1)

    # this line makes series neutral to a constant column so that it's centered and for sure gets corr 0 with exposures
    exposures = np.hstack(
        (exposures,
         np.array([np.mean(series)] * len(exposures)).reshape(-1, 1)))

    correction = proportion * (exposures.dot(
        np.linalg.lstsq(exposures, scores, rcond=None)[0]))
    corrected_scores = scores - correction
    neutralized = pd.Series(corrected_scores.ravel(), index=series.index)
    return neutralized


def unif(df):
    x = (df.rank(method="first") - 0.5) / len(df)
    return pd.Series(x, index=df.index)

def get_feature_neutral_mean(df):
    feature_cols = [c for c in df.columns if c.startswith("feature")]
    df.loc[:, "neutral_sub"] = neutralize(df, [PREDICTION_NAME],
                                          feature_cols)[PREDICTION_NAME]
    scores = df.groupby("era").apply(
        lambda x: np.corrcoef(x["neutral_sub"].rank(pct=True, method="first"), x[TARGET_NAME])).mean()
    return np.mean(scores)

def compute_val_mmc(valid_df : pd.DataFrame):    
    # MMC over validation
    mmc_scores = []
    corr_scores = []
    for _, x in valid_df.groupby("era"):
        series = neutralize_series(pd.Series(unif(x[PREDICTION_NAME])),
                                   pd.Series(unif(x[EXAMPLE_PRED])))
        mmc_scores.append(np.cov(series, x[TARGET_NAME])[0, 1] / (0.29 ** 2))
        corr_scores.append(np.corrcoef(unif(x[PREDICTION_NAME]).rank(pct=True, method="first"), x[TARGET_NAME]))

    val_mmc_mean = np.mean(mmc_scores)
    val_mmc_std = np.std(mmc_scores)
    val_mmc_sharpe = val_mmc_mean / val_mmc_std
    corr_plus_mmcs = [c + m for c, m in zip(corr_scores, mmc_scores)]
    corr_plus_mmc_sharpe = np.mean(corr_plus_mmcs) / np.std(corr_plus_mmcs)
    corr_plus_mmc_mean = np.mean(corr_plus_mmcs)

    print("MMC Mean = {:.6f}, MMC Std = {:.6f}, CORR+MMC Sharpe = {:.4f}".format(val_mmc_mean, val_mmc_std, corr_plus_mmc_sharpe))

    # Check correlation with example predictions
    corr_with_example_preds = np.corrcoef(valid_df[EXAMPLE_PRED].rank(pct=True, method="first"),
                                          valid_df[PREDICTION_NAME].rank(pct=True, method="first"))[0, 1]
    print("Corr with example preds: {:.4f}".format(corr_with_example_preds))
    
    return val_mmc_mean, val_mmc_std, corr_plus_mmc_sharpe, corr_with_example_preds


# this is the main method. The rest are just called interanlly. 
def score_summary(valid_df : pd.DataFrame):
    score_df = {}
    
    try:
        score_df['correlation'] = compute_val_corr(valid_df)
    except:
        print('ERR: computing correlation')
    try:
        score_df['corr_sharpe'], score_df['corr_mean'], score_df['corr_std'], score_df['max_drawdown'] = compute_val_sharpe(valid_df)
    except:
        print('ERR: computing sharpe')
    try:
        score_df['feature_exposure'], score_df['max_feature_exposure'] = compute_val_feature_exposure(valid_df)
    except:
        print('ERR: computing feature exposure')
    try:
        score_df['mmc_mean'], score_df['mmc_std'], score_df['corr_mmc_sharpe'], score_df['corr_with_example_xgb'] = compute_val_mmc(valid_df)
    except:
        print('ERR: computing MMC')
    
    return pd.DataFrame.from_dict(score_df, orient='index')

In [30]:
# # prediction for valid periods   
# # peek at the number of prediction
# pred = model.predict(valid[features])
# print(f'You made {len(pred)} total predictions')
# print(type(pred))
# print(pred[0])

In [31]:
# # scores


# valid_df = valid4score(valid, pred, load_example=False, save=False)

# score_df = pd.DataFrame()
# print('------------------')
# print('ALL:')
# print('------------------')
# all_ = score_summary(valid_df).rename(columns={0: 'all'})

# print('------------------')
# print('VALID 1:')
# print('------------------')
# val1_ = score_summary(valid_df.query('era < 150')).rename(columns={0: 'val1'}) 
# # there might be something strange with these numbers. before they are 180 need to verify unsure what valid2 = True means.

# print('------------------')
# print('VALID 2:')
# print('------------------')
# val2_ = score_summary(valid_df.query('era > 150')).rename(columns={0: 'val2'})

# See Scores

In [32]:
# # scores
# score_df = pd.concat([all_, val1_, val2_], axis=1)
# score_df.style.background_gradient(cmap='viridis', axis=0)

# Submission


1. Create the prediction list.
2. Link those predictions with the tournment data
3. Write the id, prediction to a csv file.
4. Use numerai wrapper to submit that .csv file as your current model



### Methods to handle submission

In [33]:
def load_api_creds_into_dict(): # works
  """
    Read creds.json and return a dictionary of your API keys.
  """
  creds  = open('creds.json','r')
  api_keys_dict = json.load(creds)
  creds.close()
  return api_keys_dict


def open_api_access(): # works
    """
    Read in my private key from creds.json and return the numerai api wrapper
    """
    api_keys_dict = load_api_creds_into_dict()
    my_secret_key = api_keys_dict['secret_key']
    my_public_id = api_keys_dict['public_id']
    napi = numerapi.NumerAPI(secret_key=my_secret_key, public_id=my_public_id)
    return napi


def create_id_prediction_df(tournament_data: pd.DataFrame, model_predictions : np.ndarray): # works
    """
    Create a dataframe that looks like 
    id,prediction
    asdfads,.5429
    asdfaddsss,.5051
    ...
    """
    predictions_df = tournament_data["id"].to_frame() # get all the Ids and cast them to a Dataframe
    predictions_df[PREDICTION_NAME] = model_predictions #add your predictions to the id frame
    return predictions_df # data frame of id, prediction


def write_predictions_to_file(prediction_df: pd.DataFrame): # works
    my_file_name = 'myPredictions.csv'
    try:
      out_location = open(my_file_name, 'x')
    except:
      out_location = open(my_file_name, 'w')

    prediction_df.to_csv(out_location, index=False)
    out_location.close()
    return my_file_name 


def run_model_and_create_prediction_file(model_object, tournament_data: pd.DataFrame, features: list): # untested
  """
    This stitches everything together.

    Pass it a trained model and the tournament data set, the list of feature columns
    1. Does preditions
    2. write the predictions to a file.
    3. returns the name fo the file where my predictions are saved data is saved
  """
  model_predictions = model_object.predict(tournament_data[features])
  prediction_df = create_id_prediction_df(tournament_data,model_predictions)
  file_with_predictions = write_predictions_to_file(prediction_df)
  return file_with_predictions


def submit_predictions_to_numerai(filename_of_predictions, sumbit_model_id):
    napi = open_api_access() # open a connection to the numerai API with your creds.json file
    submission_id = napi.upload_predictions(filename_of_predictions, model_id=sumbit_model_id)
    print(f'You successfully submitted for {sumbit_model_id}')
    print(type(submission_id))
    return submission_id

print('your helper methods work correctly')
    

your helper methods work correctly


In [35]:
api_keys_dict = load_api_creds_into_dict()
mrquantsalot_model_id = api_keys_dict['mr_quants_model_id']
tut_model_model_id = api_keys_dict['tutmodel_model_id']
PREDICTION_NAME = "prediction" # this is the header of the csv file you are creating
OUTPUT_DIR = '' # just the root of your local folder in this instance of google colab

### Run your model on the real tournament data.

## Run and submit MRQUANTSALOT


In [36]:
%%time
mr_quants_file_with_predictions = run_model_and_create_prediction_file(mr_quants_model, tournament_data, features)
numerai_submission_id_mrQ = submit_predictions_to_numerai(mr_quants_file_with_predictions, mrquantsalot_model_id)

2021-03-23 21:55:17,761 INFO numerapi.base_api: uploading predictions...
2021-03-23 21:55:23,774 ERROR numerapi.base_api: Can't update submission after deadline if previous submission was on time


ValueError: ignored

## Run and submit TUT_MODEL

In [37]:
tut_model_file_with_predictions = run_model_and_create_prediction_file(tut_model, tournament_data, features)
numerai_submission_id_tut = submit_predictions_to_numerai(tut_model_file_with_predictions, tut_model_model_id)

2021-03-23 22:04:40,134 INFO numerapi.base_api: uploading predictions...
2021-03-23 22:04:46,269 ERROR numerapi.base_api: Can't update submission after deadline if previous submission was on time


ValueError: ignored